In [36]:
import glob
import os
import numpy as np

loading data n/
try imdb load data

In [40]:
#cat_file = os.path.join("/Users/jean-baptiste/code/Sythak/deepdraw/notebooks/ndjson_simplified", 'full_simplified_cat.ndjson')

In [42]:
#cat1 = np.load(cat_file)
with open('./cat.ndjson', 'r') as cats_ndjson_file :
    cats_simp_ndjson_lines = cats_ndjson_file.readlines()

FileNotFoundError: [Errno 2] No such file or directory: './cat.ndjson'

In [24]:
cat1

{'word': 'cat',
 'countrycode': 'VE',
 'timestamp': '2017-03-02 23:25:10.07453 UTC',
 'recognized': True,
 'key_id': '5201136883597312',
 'drawing': [[[130,
    113,
    99,
    109,
    76,
    64,
    55,
    48,
    48,
    51,
    59,
    86,
    133,
    154,
    170,
    203,
    214,
    217,
    215,
    208,
    186,
    176,
    162,
    157,
    132],
   [72,
    40,
    27,
    79,
    82,
    88,
    100,
    120,
    134,
    152,
    165,
    184,
    189,
    186,
    179,
    152,
    131,
    114,
    100,
    89,
    76,
    0,
    31,
    65,
    70]],
  [[76, 28, 7], [136, 128, 128]],
  [[76, 23, 0], [160, 164, 175]],
  [[87, 52, 37], [175, 191, 204]],
  [[174, 220, 246, 251], [134, 132, 136, 139]],
  [[175, 255], [147, 168]],
  [[171, 208, 215], [164, 198, 210]],
  [[130, 110, 108, 111, 130, 139, 139, 119],
   [129, 134, 137, 144, 148, 144, 136, 130]],
  [[107, 106], [96, 113]]]}

define X, y

In [43]:
unwanted = set(cat1) - set(['word', 'key_id', 'drawing'])
for unwanted_key in unwanted: del cat1[unwanted_key]
for key in ['word', 'key_id', 'drawing']:
    print(f"cat1['{key}'] = {cat1[key]}", end='\n\n')

cat1['word'] = cat

cat1['key_id'] = 5201136883597312

cat1['drawing'] = [[[130, 113, 99, 109, 76, 64, 55, 48, 48, 51, 59, 86, 133, 154, 170, 203, 214, 217, 215, 208, 186, 176, 162, 157, 132], [72, 40, 27, 79, 82, 88, 100, 120, 134, 152, 165, 184, 189, 186, 179, 152, 131, 114, 100, 89, 76, 0, 31, 65, 70]], [[76, 28, 7], [136, 128, 128]], [[76, 23, 0], [160, 164, 175]], [[87, 52, 37], [175, 191, 204]], [[174, 220, 246, 251], [134, 132, 136, 139]], [[175, 255], [147, 168]], [[171, 208, 215], [164, 198, 210]], [[130, 110, 108, 111, 130, 139, 139, 119], [129, 134, 137, 144, 148, 144, 136, 130]], [[107, 106], [96, 113]]]



Encoding raw --> lines

In [ ]:
def raw_to_lines(raw):
    """Convert QuickDraw raw format into polyline format."""
    result = []
    N = len(raw)
    for i in range(N):
        line = []
        rawline = raw[i]
        M = len(rawline[0])
        #if M <= 2:
            #continue
        for j in range(M):
            line.append([rawline[0][j], rawline[1][j]]) 
        result.append(line)
    return result

cat1_lines = raw_to_lines(cat1['drawing'])
cat1_lines

Encoding lines --> strokes_3

In [ ]:
def lines_to_strokes(lines, delta=True):
    """Convert polyline format to stroke-3 format. x-offset, y-offset is optional."""
    eos = 0
    strokes = [[0, 0, 0]]
    for line in lines:
        linelen = len(line)
        for i in range(linelen):
            eos = 0 if i < linelen - 1 else 1
            strokes.append([line[i][0], line[i][1], eos])
    strokes = np.array(strokes)
    if delta :
        strokes[1:, 0:2] -= strokes[:-1, 0:2] # Compute deltas
    return strokes[1:, :] # Trunc the first point

cat1_strokes_3 = lines_to_strokes(cat1_lines, delta=True)
cat1_strokes_3